In [1]:
from pyvi.pyvi import ViTokenizer,ViPosTagger

def readFile_txt(filename):
    data=[]
    f = open(filename)
#     next(f)
    for row in f:
        data.append(row)
    return data

def make_data_train(data):
    data_train=[]
    for row in data:
        post_tup=ViPosTagger.postagging(ViTokenizer.tokenize(row))
        post_list=tupToList(post_tup)
        data_train.append(post_list)
    return data_train

def writeListFile(filename,data):
    str_out=''
    for item in data:
        for tup in item:
            str_out+=tup[0]+" "+tup[1]+" "+tup[2]+"\n"
        str_out+="\n"
#     print(str_out)
    F = open(filename,"w")
    F.write(str(str_out))
def tupToList(tup):
    list1=[]
    for item in tup[0]:
        index=tup[0].index(item)
        tup_=(item,tup[1][index],'0')
        list1.append(tup_)
    return list1

In [2]:

data=readFile_txt("train1")
data_train=make_data_train(data)

writeListFile("train1.pre",data_train)

data=readFile_txt("test1")
data_test=make_data_train(data)

writeListFile("test1.pre",data_test)

In [3]:
# create data train
file_train="train2.pre"
file_test="test2.pre"
data_train=readFile_txt(file_train)
data_test=readFile_txt(file_test)
# print(data_test)
def getMatrix(data):
    matrix=[]
    vector=[]
    line=0
    for dt in data:
        if len(dt)>1:
            dt = dt[:-1]
            l1=dt.split(" ")
            t1=(l1[0],l1[1],l1[2])
            vector.append(t1)
        else:
            matrix.append(vector)
            vector=[]
    print (len(matrix))
    return matrix
matrix_train=getMatrix(data_train)
matrix_test=getMatrix(data_test)
# print(matrix_test)

439
106


In [4]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [5]:
sent2features(matrix_train[0])[0]

{'+1:postag': 'N',
 '+1:postag[:2]': 'N',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'đất',
 'BOS': True,
 'bias': 1.0,
 'postag': 'N',
 'postag[:2]': 'N',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'giá',
 'word[-2:]': 'iá',
 'word[-3:]': 'Giá'}

In [6]:
%%time
X_train = [sent2features(s) for s in matrix_train]
y_train = [sent2labels(s) for s in matrix_train]

X_test = [sent2features(s) for s in matrix_test]
y_test = [sent2labels(s) for s in matrix_test]
# print(X_test)

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 57.6 ms


In [7]:
%%time
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
import pickle
f = open('../model/ner.pkl', 'wb')
pickle.dump(crf, f)
f.close()

CPU times: user 972 ms, sys: 52 ms, total: 1.02 s
Wall time: 2.37 s


In [8]:
labels = list(crf.classes_)
labels.remove('0')
labels

['district',
 'province',
 'street',
 'maxPrice',
 'ward',
 'unit',
 'minPrice',
 'minAcreage',
 'project',
 'bedRoom']

In [9]:
y_pred = crf.predict(X_test)
# print(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.7716975206871105

In [10]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

       ward      0.500     1.000     0.667         1
   maxPrice      0.667     1.000     0.800         4
    bedRoom      1.000     0.333     0.500         3
 minAcreage      1.000     1.000     1.000         3
   minPrice      1.000     0.500     0.667         4
   district      0.833     0.833     0.833        18
       unit      1.000     0.857     0.923         7
    project      0.750     0.500     0.600         6
   province      0.889     0.533     0.667        15
     street      0.769     1.000     0.870        10

avg / total      0.855     0.746     0.772        71



In [22]:
test_list_sentence=['thuê văn_phòng quận Cầu_Giấy , Hà_Nội , giá bao_nhiêu']
filename="sentence_test"
test_sentence=make_data_train(test_list_sentence)
# print(test_sentence)
def writeListFile(data):
    str_out=[]
    for item in data:
        for tup in item:
            str_out.append(tup[0]+" "+tup[1]+" "+tup[2]+"\n")

    return str_out
# print(writeListFile(test_sentence))
data_test=readFile_txt(filename)
# print(data_test)
matrix_test_sentence=getMatrix(data_test)
print(matrix_test_sentence)
X_matrix_test_sentence = [sent2features(s) for s in matrix_test_sentence]
y_pred = crf.predict(X_matrix_test_sentence)
y_pred
# X_matrix_test_sentence

1
[[('thuê', 'V', '0'), ('văn_phòng', 'N', '0'), ('quận', 'N', '0'), ('Cầu_Giấy', 'Np', '0'), (',', 'F', '0'), ('Hà_Nội', 'Np', '0'), (',', 'F', '0'), ('giá', 'N', '0'), ('bao_nhiêu', 'A', '0')]]


[['0', '0', '0', 'district', '0', 'province', '0', '0', '0']]